In [1]:
import os
import sys
from pathlib import Path
import pandas as pd
from datetime import datetime
#import sqlalchemy as sa

In [3]:
sys.path.append(r"../../src/collectcube")
from db import *
from db_post import *

In [4]:
out_dir = "C:/GISprojects/ParaguayTraining"
local_db_path = os.path.join(out_dir, 'py_training.db')

In [5]:
table_check = check_table(local_db_path,'pixels')

              PID  Center     cent_X     cent_Y   cent_lat  cent_long  \
277510  0031482_4       0  3081000.0 -3094060.0 -24.458177 -56.400685   
277511  0031482_5       0  3081020.0 -3094060.0 -24.458177 -56.400467   
277512  0031482_6       0  3081000.0 -3094070.0 -24.458259 -56.400675   
277513  0031482_7       0  3081010.0 -3094070.0 -24.458259 -56.400566   
277514  0031482_8       0  3081020.0 -3094070.0 -24.458259 -56.400457   

        ransamp  checked   PID0  PID1 sampgroup      rand     rand2 biome  \
277510        1        0  31482     4  GE_extra  0.564274  0.539983    BH   
277511        1        0  31482     5  GE_extra  0.564274  0.539983    BH   
277512        1        0  31482     6  GE_extra  0.564274  0.539983    BH   
277513        1        0  31482     7  GE_extra  0.564274  0.539983    BH   
277514        1        0  31482     8  GE_extra  0.564274  0.539983    BH   

        gridCell  
277510      3867  
277511      3867  
277512      3867  
277513      3867  
277

## Open db table as pandas dataframe

## Get sample for target date (direct and indirect)

In [6]:
## Will consider anything 1-June of prior year to 1-June of target_date year as direct observation 
## and interpolate from other observations according to the rules set up and the 'stability' column in LUT
## if sample_type is 'area_est' or 'validation', will only include center pixels

target_yr = 2022

sample_type = 'training' ## 'area_est',  validation', 'training' 
sample_cats = 'LC25'   ## 'bi_tree', 'bi_forest'
lut = '../../data/Class_LUT.csv'
   
all_obs=get_sample_for_date(target_yr,local_db_path,sample_type,sample_cats,lut)
    
all_obs.sort_values('PID0')
all_obs.tail(n=10)

there are 27856 observations for selected date


,recID,PID0,PID1,imgDate,LC5,LC,HOMONBHD9,ForestProx,WaterProx,PercentTree,...,cent_lat,cent_long,ransamp,checked,sampgroup,rand,rand2,biome,gridCell,LC2
27846,93066.0,31454,0,2023-06-06,20,26,3.0,NaN,NaN,NaN,...,-24.090690,-56.546396,0,0,GE_extra,0.692447,0.825737,BH,3829,0
27847,93071.0,31454,1,2023-06-06,20,26,5.0,NaN,NaN,NaN,...,-24.090609,-56.546515,0,0,GE_extra,0.692447,0.825737,BH,3829,0
27848,93075.0,31455,0,2023-06-06,70,66,8.0,NaN,NaN,NaN,...,-24.102268,-56.543379,0,0,GE_extra,0.988693,0.472295,BH,3829,0
27849,93143.0,31466,0,2022-01-01,70,54,8.0,NaN,NaN,NaN,...,-24.694270,-56.040257,0,0,GE_extra,0.542753,0.451004,BH,3904,0
27850,93150.0,31467,0,2022-01-01,70,54,7.0,NaN,NaN,NaN,...,-24.697948,-56.042310,0,0,GE_extra,0.753376,0.282749,BH,3904,0
27851,93157.0,31468,0,2023-03-30,40,46,8.0,NaN,NaN,NaN,...,-24.701217,-56.044631,0,0,GE_extra,0.853632,0.675750,BH,3904,0
27852,93161.0,31469,0,2023-10-01,70,66,8.0,NaN,NaN,NaN,...,-24.691327,-55.999912,0,0,GE_extra,0.721903,0.128903,BH,3904,0
27853,93170.0,31472,0,2021-06-12,40,41,8.0,NaN,NaN,NaN,...,-27.239530,-55.813895,1,0,GE_extra,0.907977,0.050668,BH,3919,0
27854,93177.0,31473,0,2021-06-12,20,31,8.0,NaN,NaN,NaN,...,-27.239860,-55.811204,1,0,GE_extra,0.972399,0.391651,BH,3919,0
27855,93182.0,31474,0,2021-06-12,40,41,8.0,NaN,NaN,NaN,...,-27.240026,-55.813605,1,0,GE_extra,0.356496,0.106146,BH,3919,0


In [5]:
#all_obs.to_csv(os.path.join(out_dir,'2024obscheck.csv'))

## Balance sample by class and quality

In [8]:
lut=Path('../../data/Class_LUT.csv')
#print(pd.read_csv(lut).sort_values('LC_UNQ')[['LC_UNQ','USE_NAME','LC25','LC25_name','LC3_name']])
cutoff = 300
mixbal = 4
dfbal = balance_training_data(target_yr, lut, all_obs, out_dir, cutoff, mixbal)

tot percent =: 0.98813
             LC25_name  counts
0   Crops-SoyCornWheat    4517
1           Mixed-path    2763
2            Crops-mix    2676
3    Grassland-Managed    2108
4         Trees-Forest    2076
5        Mixed-VegEdge    1965
6              unknown    1575
7        Grassland-Wet    1293
8    Grassland-Natural    1230
9      Trees-disturbed    1160
10           TreePlant     803
11               Shrub     687
12    Crops-Yerba-Mate     639
13         NoVeg_Built     612
14         NoVeg_Water     500
15      Grass_tree-mix     467
16          NoVeg_Bare     451
17  Trees-Forest_shrub     287
18        Crops-Banana     269
19   Trees-Forest_palm     258
20         Crops-Sugar     222
21       Crops-Orchard     216
22             Cleared     157
23          Crops-Rice     142
24      Crops-Mandioca     109
25           Trees-wet     106
26        Crops-Sesame      77
27         Burnt-woody      74
28        Crops-Cotton      65
29       Crops-Shrub47      54
30  Crops-hortic

## Summarize sample size by sample group